# BMP를 CSV 파일로 변환


In [3]:
import os
import csv
from PIL import Image
import numpy as np

def process_bmp_folder(input_folder, output_csv):
    """
    BMP 파일이 여러 폴더에 저장되어 있는 데이터를 읽어와 CSV 파일로 변환합니다.
    빈 데이터는 -999로 대체하여 처리합니다.

    Args:
        input_folder (str): BMP 파일이 있는 최상위 폴더 경로
        output_csv (str): 생성될 CSV 파일 경로
    """
    # CSV 파일 열기
    with open(output_csv, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # 각 폰트 폴더를 탐색
        for font_folder in os.listdir(input_folder):
            font_path = os.path.join(input_folder, font_folder)

            # 폰트 폴더인지 확인
            if not os.path.isdir(font_path):
                continue

            print(f"Processing folder: {font_folder}")

            i = 0
            # BMP 파일 탐색
            for bmp_file in sorted(os.listdir(font_path)):
                if not bmp_file.endswith('.bmp'):
                    continue

                target_value = i

                # BMP 파일 읽기
                bmp_path = os.path.join(font_path, bmp_file)
                img = Image.open(bmp_path).convert('L')  # Grayscale 변환
                img = img.resize((28, 28), Image.Resampling.LANCZOS)  # 28x28로 크기 조정
                img_data = np.asarray(img).flatten()  # 1D 배열로 변환

                # 입력 데이터 스케일링 (0.01 ~ 1.0)
                scaled_data = 1.0 - (img_data / 255.0 * 0.99) + 0.01

                # 결측값 처리: 잘못된 값이나 빈 값을 -999로 대체
                scaled_data = [float(x) if x != '' else -999 for x in scaled_data]

                # CSV 한 줄 구성: [타겟값, 입력 데이터...]
                row = [float(target_value)] + scaled_data
                csv_writer.writerow(row)
                i += 1

        print(f"CSV 파일 생성 완료: {output_csv}")

# 사용 예시
input_folder = "../Images"  # BMP 파일이 있는 최상위 폴더 경로
output_csv = "../Data/font_data_1540.csv"  # 생성될 CSV 파일 경로
process_bmp_folder(input_folder, output_csv)

Processing folder: hy얕은샘물m-yoond1004
Processing folder: hy궁서-yoond1004
Processing folder: HANBatang
Processing folder: malgun
Processing folder: H2HDRM
Processing folder: hy엽서m-yoond1004
Processing folder: H2GTRE
Processing folder: HANDOTUMB
Processing folder: HanSantteutDotum-Regular
Processing folder: Hancom Gothic Regular
CSV 파일 생성 완료: ../Data/font_data_1540.csv


# csv 파일에서 값 들고와서 plot 그려보기
데이터 제대로 생성됐는지 확인하기


In [9]:
import csv
import numpy as np
import matplotlib.pyplot as plt

def plot_csv_data(csv_file, row_index=0):
    """
    CSV 파일에서 특정 행의 데이터를 28x28 그레이스케일 플롯으로 시각화합니다.
    
    Args:
        csv_file (str): CSV 파일 경로
        row_index (int): 시각화할 행의 인덱스 (기본값: 0번째 행)
    """
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        rows = list(reader)
        
        # 선택한 행 가져오기
        selected_row = rows[row_index]
        
        # 0번째 값 (타겟값) 출력
        target_value = selected_row[0]
        print(f"Target Value: {target_value}")
        
        # 1번째 값부터 끝까지 가져와서 64x64로 변환
        pixel_data = np.array(selected_row[0:], dtype=float).reshape((64,64))
        
        # 이미지 플롯
        plt.imshow(pixel_data, cmap='gray')
        plt.title(f"Gray Plot (Target: {target_value})")
        plt.colorbar()
        plt.show()

# 사용 예시
csv_file = "../Data/font_data.csv"  # 생성한 CSV 파일 경로
plot_csv_data(csv_file, row_index=81)


FileNotFoundError: [Errno 2] No such file or directory: '../Data/font_data.csv'

In [7]:
import os
import csv
from PIL import Image
import numpy as np

def process_bmp_folder(input_folder, output_csv):
    """
    BMP 파일이 여러 폴더에 저장되어 있는 데이터를 읽어와 CSV 파일로 변환합니다.
    각 글자 폴더에 대해 순서대로 타겟값(0부터 시작)을 할당합니다.

    Args:
        input_folder (str): BMP 파일이 있는 최상위 폴더 경로
        output_csv (str): 생성될 CSV 파일 경로
    """
    # CSV 파일 열기
    with open(output_csv, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # 각 폰트 폴더 탐색
        target_value = 0
        for font_folder in sorted(os.listdir(input_folder)):
            font_path = os.path.join(input_folder, font_folder)
            # 각 폰트 폴더 탐색
            target_value = 0
            # 폰트 폴더인지 확인
            if not os.path.isdir(font_path):
                continue

            print(f"Processing folder: {font_folder}")

            # 각 글자 폴더 탐색
            for char_folder in sorted(os.listdir(font_path)):
                char_path = os.path.join(font_path, char_folder)

                # 글자 폴더인지 확인
                if not os.path.isdir(char_path):
                    continue

                print(f"  Processing character folder: {char_folder}")

                # BMP 파일 탐색
                for bmp_file in sorted(os.listdir(char_path)):
                    if not bmp_file.endswith('.bmp'):
                        continue

                    # BMP 파일 읽기
                    bmp_path = os.path.join(char_path, bmp_file)
                    img = Image.open(bmp_path).convert('L')  # Grayscale 변환
                    img = img.resize((28, 28), Image.Resampling.LANCZOS)  # 28x28로 크기 조정
                    img_data = np.asarray(img).flatten()  # 1D 배열로 변환

                    # 입력 데이터 스케일링 (0.01 ~ 1.0)
                    scaled_data = 1.0 - (img_data / 255.0 * 0.99) + 0.01

                    # CSV 한 줄 구성: [타겟값, 입력 데이터...]
                    
                    row = [float(target_value)] + list(scaled_data)
                    print(row[0])
                    csv_writer.writerow(row)

                # 타겟값 증가 (다음 글자 폴더로 이동)
                target_value += 1

        print(f"CSV 파일 생성 완료: {output_csv}")

# 사용 예시
input_folder = "../Image"  # BMP 파일이 있는 최상위 폴더 경로
output_csv = "../Data/font_data_7700.csv"  # 생성될 CSV 파일 경로
process_bmp_folder(input_folder, output_csv)


Processing folder: H2GTRE
  Processing character folder: AC00
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
  Processing character folder: AC04
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
  Processing character folder: AC11
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
  Processing character folder: AC19
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
  Processing character folder: AC70
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
  Processing character folder: AC83
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
  Processing character folder: AC8C
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
  Processing character folder: ACBD
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
  Processing character folder: ACE0
8.0
8.0
8.

KeyboardInterrupt: 